In [1]:
# importing necessary libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import urllib
import re
# from pandasql import sqldf
from lxml import html
from datetime import datetime, timedelta
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import scipy.stats
import pandas_datareader as web
import matplotlib.dates as mdates
plt.style.use('bmh')

### Suppress Warnings:
import warnings
warnings.filterwarnings('ignore')

In [6]:
!pip install xlrd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 3.2 MB/s eta 0:00:00a 0:00:01


# Loading Data

In [2]:
df_2016 = pd.read_excel("Data/native_Load_2016.xlsx")
df_2017 = pd.read_excel("Data/native_Load_2017.xlsx")
df_2018 = pd.read_excel("Data/Native_Load_2018.xlsx")
df_2019 = pd.read_excel("Data/Native_Load_2019.xlsx")
df_2020 = pd.read_excel("Data/Native_Load_2020.xlsx")
df_2021 = pd.read_excel("Data/Native_Load_2021.xlsx")
df_2022 = pd.read_excel("Data/Native_Load_2022.xlsx")
df_2023 = pd.read_excel("Data/Native_Load_2023.xlsx")

In [3]:
df_2023.head()

,Hour Ending,COAST,EAST,FWEST,NORTH,NCENT,SOUTH,SCENT,WEST,ERCOT
0,01/01/2023 01:00,8514.637876,1218.506271,5073.526557,917.394049,9896.757586,2859.017922,6012.400469,1116.933155,35609.173887
1,01/01/2023 02:00,8345.632899,1179.885576,5016.774637,906.854582,9675.869720,2845.779210,5934.084260,1107.417972,35012.298856
2,01/01/2023 03:00,8125.706879,1151.144653,5016.303220,894.817013,9490.332365,2734.163642,5855.790279,1101.324245,34369.582295
3,01/01/2023 04:00,7995.083655,1153.306928,5038.003823,891.733538,9420.890470,2555.339795,5840.914307,1105.457821,34000.730337
4,01/01/2023 05:00,7906.920393,1168.779618,4994.502755,896.893843,9430.780728,2484.777679,5873.683697,1108.739956,33865.078669


In [4]:
df_2016 = df_2016.rename(columns={"FAR_WEST": "FWEST", "NORTH_C":"NCENT", "SOUTHERN":"SOUTH", "SOUTH_C":"SCENT"})
df_2017 = df_2017.rename(columns={"Hour Ending": "Hour_End"})
df_2018 = df_2018.rename(columns={"HourEnding": "Hour_End"})
df_2019 = df_2019.rename(columns={"HourEnding": "Hour_End"})
df_2020 = df_2020.rename(columns={"HourEnding": "Hour_End"})
df_2021 = df_2021.rename(columns={"Hour Ending": "Hour_End"})
df_2022 = df_2022.rename(columns={"Hour Ending": "Hour_End"})
df_2023 = df_2023.rename(columns={"Hour Ending": "Hour_End"})

In [23]:
df_2016.head()

,Hour_End,COAST,EAST,FWEST,NORTH,NCENT,SOUTH,SCENT,WEST,ERCOT
0,2016-01-01 01:00:00.003,9001.477932,1148.358788,2107.895986,788.543919,11409.339036,2829.993195,5479.057816,1088.091916,33852.758587
1,2016-01-01 01:59:59.997,8864.555456,1134.909662,2100.702470,782.355108,11235.174936,2830.075196,5405.955778,1080.369336,33434.097940
2,2016-01-01 03:00:00.000,8745.758728,1126.864026,2099.966167,781.237004,11151.965219,2789.470654,5334.159606,1069.644619,33099.066023
3,2016-01-01 04:00:00.003,8689.879755,1129.417663,2104.947864,785.264364,11209.701072,2757.999808,5302.938201,1073.658885,33053.807611
4,2016-01-01 04:59:59.997,8733.703094,1141.769602,2117.072608,790.248115,11428.716965,2769.678996,5364.298843,1096.410845,33441.899067


In [35]:
df_2021.head()

,Hour_End,COAST,EAST,FWEST,NORTH,NCENT,SOUTH,SCENT,WEST,ERCOT
0,01/01/2021 01:00,11047.018793,1517.435253,3978.220628,838.352810,13575.277728,3545.093310,7778.262533,1440.188560,43719.849616
1,01/01/2021 02:00,10954.470339,1523.683893,3975.757045,825.123437,13340.337915,3586.547882,7693.744119,1421.385715,43321.050347
2,01/01/2021 03:00,10883.657240,1519.800916,3969.812081,819.551228,13311.625189,3586.124059,7566.951524,1405.544826,43063.067063
3,01/01/2021 04:00,10926.482445,1523.472963,3983.487761,820.612439,13356.442066,3570.592369,7506.685527,1402.283632,43090.059203
4,01/01/2021 05:00,11019.796399,1565.625573,3980.940744,831.607158,13528.651487,3590.902242,7563.971854,1405.094618,43486.590073


In [31]:
df_2023.head()

,Hour_End,COAST,EAST,FWEST,NORTH,NCENT,SOUTH,SCENT,WEST,ERCOT
0,01/01/2023 01:00,8514.637876,1218.506271,5073.526557,917.394049,9896.757586,2859.017922,6012.400469,1116.933155,35609.173887
1,01/01/2023 02:00,8345.632899,1179.885576,5016.774637,906.854582,9675.869720,2845.779210,5934.084260,1107.417972,35012.298856
2,01/01/2023 03:00,8125.706879,1151.144653,5016.303220,894.817013,9490.332365,2734.163642,5855.790279,1101.324245,34369.582295
3,01/01/2023 04:00,7995.083655,1153.306928,5038.003823,891.733538,9420.890470,2555.339795,5840.914307,1105.457821,34000.730337
4,01/01/2023 05:00,7906.920393,1168.779618,4994.502755,896.893843,9430.780728,2484.777679,5873.683697,1108.739956,33865.078669


## Merging Data
>- Set time variable as index and change it to datetime object
>- Resampling each dataset hourly because this project is on hourly energy prediction
>- Keeping only South Central Region SCENT as it has small weather difference across the region
>- It will be possible to predict 

In [5]:
#Concatenate dataframes on rows axis
dfs = [df_2016, df_2017, df_2018, df_2019, df_2020, df_2021, df_2022, df_2023 ] 
hourly_df = pd.concat(dfs, axis='rows', ignore_index=True)

In [6]:
hourly_df.to_csv("texas_data.csv")

In [7]:
df = pd.read_csv("texas_data.csv", parse_dates=['Hour_End'], dayfirst=True)

In [11]:
df.head(1)

,Unnamed: 0,Hour_End,COAST,EAST,FWEST,NORTH,NCENT,SOUTH,SCENT,WEST,ERCOT
0,0,2016-01-01 01:00:00.003000,9001.477932,1148.358788,2107.895986,788.543919,11409.339036,2829.993195,5479.057816,1088.091916,33852.758587


In [10]:
df.tail(1)

,Unnamed: 0,Hour_End,COAST,EAST,FWEST,NORTH,NCENT,SOUTH,SCENT,WEST,ERCOT
68662,68662,10/31/2023 24:00,11142.436715,1541.092166,5825.194539,1140.022089,11749.284714,3319.485039,7037.511893,1205.836656,42960.863811


In [14]:
df.shape

(68663, 11)

In [15]:
df.isna().sum()

Unnamed: 0    0
Hour_End      0
COAST         1
EAST          1
FWEST         1
NORTH         1
NCENT         1
SOUTH         1
SCENT         1
WEST          1
ERCOT         1
dtype: int64

In [37]:
# Define the start and end points of the time sequence since our date column doesnt have hours
start_date = pd.Timestamp(year=2016, month=1, day=1, hour=1, minute=0)
end_date = pd.Timestamp(year=2023, month=10, day=31, hour=22, minute=59)

time_sequence = pd.date_range(start=start_date, end=end_date, freq='H')

df_time = pd.DataFrame({'datetime':time_sequence})

In [25]:
df_time

,datetime
0,2016-01-01 01:00:00
1,2016-01-01 02:00:00
2,2016-01-01 03:00:00
3,2016-01-01 04:00:00
4,2016-01-01 05:00:00
...,...
68657,2023-10-31 18:00:00
68658,2023-10-31 19:00:00
68659,2023-10-31 20:00:00
68660,2023-10-31 21:00:00


In [38]:
# Define a custom function to handle invalid time values 
def fix_time(time):     
    if time.hour == '24:00':         
        return pd.to_datetime('00:00') + pd.DateOffset(days=1)     
    else:         
        return time 

In [42]:
# Apply the custom function to the 'Hour_End' column
df['Hour_End'] = df['Hour_End'].apply(fix_time) 
print(df)

AttributeError: 'str' object has no attribute 'hour'

In [49]:
df

,Unnamed: 0,Hour_End,COAST,EAST,FWEST,NORTH,NCENT,SOUTH,SCENT,WEST,ERCOT
0,0,2016-01-01 01:00:00.003000,9001.477932,1148.358788,2107.895986,788.543919,11409.339036,2829.993195,5479.057816,1088.091916,33852.758587
1,1,2016-01-01 01:59:59.997000,8864.555456,1134.909662,2100.702470,782.355108,11235.174936,2830.075196,5405.955778,1080.369336,33434.097940
2,2,2016-01-01 03:00:00,8745.758728,1126.864026,2099.966167,781.237004,11151.965219,2789.470654,5334.159606,1069.644619,33099.066023
3,3,2016-01-01 04:00:00.003000,8689.879755,1129.417663,2104.947864,785.264364,11209.701072,2757.999808,5302.938201,1073.658885,33053.807611
4,4,2016-01-01 04:59:59.997000,8733.703094,1141.769602,2117.072608,790.248115,11428.716965,2769.678996,5364.298843,1096.410845,33441.899067
...,...,...,...,...,...,...,...,...,...,...,...
68658,68658,10/31/2023 20:00,12014.214506,1629.502455,5596.891573,1183.013578,12591.574700,3629.608796,7276.595631,1208.662791,45130.064031
68659,68659,10/31/2023 21:00,12085.408111,1657.452412,5782.460576,1198.160939,12761.762813,3654.928960,7458.092552,1237.176010,45835.442374
68660,68660,10/31/2023 22:00,12014.939938,1678.268084,5906.298094,1196.919892,12753.164683,3647.022833,7531.944621,1254.836460,45983.394606
68661,68661,10/31/2023 23:00,11600.811435,1614.811368,5887.279189,1167.841916,12315.877536,3513.351456,7323.080873,1234.335470,44657.389244


In [55]:
# Before filling in for hr column I'll change the date to datetime and use it as index
df['Hour_End'] = pd.to_datetime(df['Hour_End'], format="%Y-%m-%d %H:%M",dayfirst=True)
df.sort_values('Hour_End', inplace=True)

ValueError: unconverted data remains when parsing with format "%Y-%m-%d %H:%M": ":00.003000", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [98]:
# # Replace "24:00" with "00:00" in the 'Hour_End' column 
# df['Hour_End'] = df['Hour_End'].replace(pd.to_datetime('24:00'), 
#                                         pd.to_datetime('00:00') + pd.DateOffset(days=1))

In [82]:
hourly_df.isna().sum()

Hour_End    0
COAST       1
EAST        1
FWEST       1
NORTH       1
NCENT       1
SOUTH       1
SCENT       1
WEST        1
ERCOT       1
dtype: int64

In [99]:
# hourly_df['Hour_End'] = pd.to_datetime(hourly_df['Hour_End']) 

In [54]:
hourly_df 

,COAST,EAST,FAR_WEST,NORTH,NORTH_C,SOUTHERN,SOUTH_C,WEST,ERCOT,Hour_End,FWEST,NCENT,SOUTH,SCENT
0,9001.477932,1148.358788,2107.895986,788.543919,11409.339036,2829.993195,5479.057816,1088.091916,33852.758587,NaN,NaN,NaN,NaN,NaN
1,8864.555456,1134.909662,2100.702470,782.355108,11235.174936,2830.075196,5405.955778,1080.369336,33434.097940,NaN,NaN,NaN,NaN,NaN
2,8745.758728,1126.864026,2099.966167,781.237004,11151.965219,2789.470654,5334.159606,1069.644619,33099.066023,NaN,NaN,NaN,NaN,NaN
3,8689.879755,1129.417663,2104.947864,785.264364,11209.701072,2757.999808,5302.938201,1073.658885,33053.807611,NaN,NaN,NaN,NaN,NaN
4,8733.703094,1141.769602,2117.072608,790.248115,11428.716965,2769.678996,5364.298843,1096.410845,33441.899067,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68658,12014.214506,1629.502455,NaN,1183.013578,NaN,NaN,NaN,1208.662791,45130.064031,10/31/2023 20:00,5596.891573,12591.574700,3629.608796,7276.595631
68659,12085.408111,1657.452412,NaN,1198.160939,NaN,NaN,NaN,1237.176010,45835.442374,10/31/2023 21:00,5782.460576,12761.762813,3654.928960,7458.092552
68660,12014.939938,1678.268084,NaN,1196.919892,NaN,NaN,NaN,1254.836460,45983.394606,10/31/2023 22:00,5906.298094,12753.164683,3647.022833,7531.944621
68661,11600.811435,1614.811368,NaN,1167.841916,NaN,NaN,NaN,1234.335470,44657.389244,10/31/2023 23:00,5887.279189,12315.877536,3513.351456,7323.080873


In [100]:
# #change the date to datetime and use it as index
# dfs = [df_2016, df_2017, df_2018, df_2019, df_2020, df_2021, df_2022, df_2023 ]  

# for df in dfs:        
#     df['Hour_End'] = pd.to_datetime(df['Hour_End'])              
#     df.set_index('Hour_End', inplace=True) 

In [101]:
# df_2015 = df_2015.resample('H').mean() 
# df_2016 = df_2016.resample('H').mean() 
# df_2017 = df_2017.resample('H').mean() 
# df_2018 = df_2018.resample('H').mean() 
# df_2019 = df_2019.resample('H').mean() 
# df_2020 = df_2020.resample('H').mean() 
# df_2021 = df_2021.resample('H').mean() 
# df_2022 = df_2022.resample('H').mean() 
# df_2023 = df_2023.resample('H').mean() 